In [ ]:
# op1
# data = data.assign(datetime = pd.to_datetime(data['Date'], format="%d/%m/%Y"))

# op2
# data["datetime"] = pd.to_datetime(data['Date'], format="%d/%m/%Y")

In [ ]:
# data = data.set_index('datetime').drop(['Day', 'Date'], axis=1)

In [ ]:
# # pandas quick plotting method
# data['totalU'].plot()

In [ ]:
# previous way of preparing training and testing data

In [ ]:
def prepare_training_data(df):

    y = data[['totalU']].to_numpy().reshape(-1, 1)
    y = y - y.mean()
    X = (df.index.year + df.index.month / 12).to_numpy().reshape(-1, 1)
    
    # change unit to million
    y = y / 1000_000
    
    return X, y

In [ ]:
X, y = prepare_training_data(data)

In [ ]:
# # test the plotting func below using one month
# monthly_data = data[data['Year'] == 2013]
# monthly_data.shape

# fig, ax = plt.subplots(figsize=(4,3))
# year_average_plot(ax, monthly_data, title='2013')

## tensorflow modelling

In [ ]:
''' tensorflow stuff '''

# import tensorflow as tf
# import tensorflow_probability as tfp
# tfd = tfp.distributions
# tfpl = tfp.layers

# print('TF version:', tf.__version__)
# print('TFP version:', tfp.__version__)

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.losses import MeanSquaredError
# from tensorflow.keras.optimizers import RMSprop
# from tensorflow import keras
# from tensorflow.keras import layers

In [ ]:
# from probabilistic_models_archive.prior import sg_prior, gmm_prior
# from probabilistic_models_archive.posterior import posterior
# from probabilistic_models_archive.utils import nll

#### SciLearn implementation

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [ ]:
kernel = 1 * RBF()
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

In [ ]:
gaussian_process.fit(X, y)

In [ ]:
mean_prediction, std_prediction = gaussian_process.predict(X_test, return_std=True)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(X, y, color="black", linestyle="dashed", label="gt", marker='.')

plt.plot(X_test, mean_prediction, label="Mean prediction")

plt.fill_between(
    X_test.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    color="tab:orange",
    alpha=0.5,
    label=r"95% confidence interval",
)

plt.show()

### Bayesian NN 

In [ ]:
x_train = X
y_train = y

In [ ]:
plt.figure(figsize=(10, 4))
plt.scatter(x_train, y_train, alpha=0.2)
plt.show()

In [ ]:
def build_and_compile_model():
    model = keras.Sequential([
      layers.Dense(64, activation='sigmoid'),
      layers.Dense(128, activation='sigmoid'),
      layers.Dense(1)
    ])

    model.compile(loss='mse',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))
    return model

In [ ]:
DNN_model = build_and_compile_model()

In [ ]:
history = DNN_model.fit(
    x_train,
    y_train,
    # validation_split=0.2,
    verbose=0, 
    batch_size=16,
    epochs=500)

In [ ]:
# plot_loss(history)

In [ ]:
test_predictions = DNN_model.predict(X_test)

In [ ]:
pseudo_train_preds = DNN_model.predict(x_train)

In [ ]:
test_predictions.shape

In [ ]:
# Plot an ensemble of trained probabilistic regressions

plt.scatter(x_train, y_train, marker='.', alpha=0.2, label='data')
plt.scatter(X_test, test_predictions)
plt.scatter(x_train, pseudo_train_preds)
# for _ in range(5):
#     y_model = model_BNN(x_train)
#     y_hat = y_model.mean()
#     y_hat_m2sd = y_hat - 2 * y_model.stddev()
#     y_hat_p2sd = y_hat + 2 * y_model.stddev()
#     if _ == 0:
#         plt.plot(x_train, y_hat, color='red', alpha=0.8, label='model $\mu$')
#         plt.plot(x_train, y_hat_m2sd, color='green', alpha=0.8, label='model $\mu \pm 2 \sigma$')
#         plt.plot(x_train, y_hat_p2sd, color='green', alpha=0.8)
#     else:
#         plt.plot(x_train, y_hat, color='red', alpha=0.8)
#         plt.plot(x_train, y_hat_m2sd, color='green', alpha=0.8)
#         plt.plot(x_train, y_hat_p2sd, color='green', alpha=0.8)        
# plt.legend()
# plt.show()

#### BNN

In [ ]:
# Create probabilistic regression with one hidden layer, weight uncertainty

model_BNN = Sequential([
    tfpl.DenseVariational(units=64,
                          input_shape=(1,),
                          make_prior_fn=sg_prior,
                          make_posterior_fn=posterior,
                          kl_weight=1/x_train.shape[0],
                          activation='sigmoid'),
    tfpl.DenseVariational(units=64,
                          input_shape=(1,),
                          make_prior_fn=sg_prior,
                          make_posterior_fn=posterior,
                          kl_weight=1/x_train.shape[0],
                          activation='sigmoid'),
    tfpl.DenseVariational(units=tfpl.IndependentNormal.params_size(1),
                          make_prior_fn=sg_prior,
                          make_posterior_fn=posterior,
                          kl_weight=1/x_train.shape[0]),
    tfpl.IndependentNormal(1)
])

model_BNN.compile(loss=nll, optimizer=RMSprop())

In [ ]:
# # Train the model

# model_BNN.fit(x_train, y_train, epochs=500, verbose=1)
# model_BNN.evaluate(x_train, y_train)

In [ ]:
# Plot an ensemble of trained probabilistic regressions

plt.scatter(x_train, y_train, marker='.', alpha=0.2, label='data')
for _ in range(5):
    y_model = model_BNN(x_train)
    y_hat = y_model.mean()
    y_hat_m2sd = y_hat - 2 * y_model.stddev()
    y_hat_p2sd = y_hat + 2 * y_model.stddev()
    if _ == 0:
        plt.plot(x_train, y_hat, color='red', alpha=0.8, label='model $\mu$')
        plt.plot(x_train, y_hat_m2sd, color='green', alpha=0.8, label='model $\mu \pm 2 \sigma$')
        plt.plot(x_train, y_hat_p2sd, color='green', alpha=0.8)
    else:
        plt.plot(x_train, y_hat, color='red', alpha=0.8)
        plt.plot(x_train, y_hat_m2sd, color='green', alpha=0.8)
        plt.plot(x_train, y_hat_p2sd, color='green', alpha=0.8)        
plt.legend()
plt.show()